In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

# List of PyTorch optimizers
optimizers = [
    optim.SGD,               # Stochastic Gradient Descent
    optim.Adam,              # Adam optimizer
    optim.AdamW,             # AdamW optimizer
    optim.Adagrad,           # Adagrad optimizer
    optim.Adadelta,          # Adadelta optimizer
    optim.RMSprop,           # RMSprop optimizer
    optim.Adamax,            # Adamax optimizer
    optim.ASGD,              # Averaged Stochastic Gradient Descent
    optim.LBFGS,             # Limited-memory BFGS
    optim.Rprop,             # Resilient backpropagation
    optim.SparseAdam,        # Sparse Adam optimizer
    optim.NAdam,             # Nesterov-accelerated Adaptive Moment Estimation
    optim.RAdam,             # Rectified Adam optimizer
]
def train_model(model, train_data, train_labels, epochs=10, learning_rate=0.001, optim_index=1):
    # Set the model to training mode
    model.train()
    train_labels = train_labels.view(-1, 1)
    # Define loss function and optimizer
    criterion = torch.nn.BCELoss()  # Use BCE for binary output (CrossEntropy if multiple classes)
    optimizer = optimizers[optim_index](model.parameters(), lr=learning_rate)

    # Convert data to PyTorch tensors
    dataset = TensorDataset(train_data, train_labels)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    # Training loop
    for epoch in range(epochs):
        epoch_loss = 0.0
        for inputs, labels in dataloader:
            # Zero gradients from the previous step
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()

            # Update weights
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

import torch
import torch.nn as nn
import numpy as np

class FFN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FFN, self).__init__()
        layers = []
        sizes = [input_size] + hidden_sizes

        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if i < len(sizes) -  1:  # No activation for the final layer
                layers.append(nn.ReLU())
        layers.append(nn.Linear(sizes[-1], 1))
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def get_parameters(self):
        """Retrieve all model parameters as a list of tensors."""
        return [p.clone().detach() for p in self.parameters()]

    def set_parameters(self, parameters):
        """Set model parameters from a list of tensors."""
        with torch.no_grad():
            for param, new_param in zip(self.parameters(), parameters):
                param.copy_(new_param)
    def extract_features(self, x):
        """Get the feature representation from the last hidden layer"""
        _, features = self.forward(x, return_features=True)
        return features[-1]  # Return the last hidden layer activation

# Create an initial population of 20 FFNs
population = [FFN(98, [20, 15], 2) for _ in range(20)]


In [2]:
def select_dataset(dataset_name):
  fairness_to_select = None
  if dataset_name == 'compas':

    import pandas as pd
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline

    # URL to the COMPAS dataset
    url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"

    # Load the dataset
    df_compas = pd.read_csv(url)
    """
    # Display the first few rows of the dataset


    # Identify categorical columns
    categorical_columns = df_compas.select_dtypes(include=['object', 'category']).columns.tolist()

    # Initialize the LabelEncoder and OneHotEncoder
    label_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)

    # Apply Label Encoding to binary categorical columns
    binary_categorical_cols = ['sex', 'score_text']  # Example binary columns
    for col in binary_categorical_cols:
        if col in categorical_columns:
            df_compas[col] = label_encoder.fit_transform(df_compas[col])

    # Apply One-Hot Encoding to other categorical columns
    other_categorical_cols = [col for col in categorical_columns if col not in binary_categorical_cols]

    # Use ColumnTransformer to apply one-hot encoding
    preprocessor = ColumnTransformer(
        transformers=[
            ('onehot', one_hot_encoder, other_categorical_cols)
        ],
        remainder='passthrough'
    )

    # Apply the transformations
    df_compas_encoded = preprocessor.fit_transform(df_compas)

    # Convert the result to a DataFrame
    df_compas_encoded = pd.DataFrame(df_compas_encoded, columns=preprocessor.get_feature_names_out())

    # Display the first few rows of the encoded dataset


    """
    df_compas = df_compas.drop('id',  axis=1)
    df_compas = df_compas.drop('dob',  axis=1)
    df_compas = df_compas.drop('compas_screening_date',  axis=1)
    df_compas = df_compas.drop('c_case_number',  axis=1)
    df_compas = df_compas.drop('screening_date', axis=1)
    df_compas = df_compas.drop('in_custody', axis=1)
    df_compas = df_compas.drop('out_custody', axis=1)
    df_compas = df_compas.drop('v_screening_date', axis=1)
    df_compas = df_compas.drop('c_offense_date', axis=1)
    df_compas = df_compas.drop('c_jail_out', axis=1)
    df_compas = df_compas.drop('c_jail_in', axis=1)
    from sklearn.preprocessing import LabelEncoder

    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()

    # Apply Label Encoding to a specific column
    df_compas['name'] = label_encoder.fit_transform(df_compas['name'])
    df_compas['first'] = label_encoder.fit_transform(df_compas['first'])
    df_compas['last'] = label_encoder.fit_transform(df_compas['last'])
    df_compas['sex'] = label_encoder.fit_transform(df_compas['sex'])
    df_compas['score_text'] = label_encoder.fit_transform(df_compas['score_text'])
    df_compas['v_type_of_assessment'] = label_encoder.fit_transform(df_compas['v_type_of_assessment'])
    df_compas['c_charge_desc'] = label_encoder.fit_transform(df_compas['c_charge_desc'])

    df_compas['r_case_number'] = label_encoder.fit_transform(df_compas['r_case_number'])
    df_compas['c_arrest_date'] = label_encoder.fit_transform(df_compas['c_arrest_date'])
    df_compas['r_charge_degree'] = label_encoder.fit_transform(df_compas['r_charge_degree'])
    df_compas['vr_case_number'] = label_encoder.fit_transform(df_compas['vr_case_number'])
    df_compas['vr_charge_degree'] = label_encoder.fit_transform(df_compas['vr_charge_degree'])
    df_compas['vr_offense_date'] = label_encoder.fit_transform(df_compas['vr_offense_date'])
    df_compas['vr_charge_desc'] = label_encoder.fit_transform(df_compas['vr_charge_desc'])
    df_compas['r_offense_date'] = label_encoder.fit_transform(df_compas['r_offense_date'])
    df_compas['r_charge_desc'] = label_encoder.fit_transform(df_compas['r_charge_desc'])
    df_compas['r_jail_in'] = label_encoder.fit_transform(df_compas['r_jail_in'])
    df_compas['r_jail_out'] = label_encoder.fit_transform(df_compas['r_jail_out'])

    import pandas as pd
    df_compas_encoded = pd.get_dummies(df_compas, columns = ['age_cat', 'race', 'c_charge_degree', 'type_of_assessment', 'v_score_text'])

    df_compas_encoded = df_compas_encoded.fillna(0)

    import torch
    import torch.nn as nn
    import numpy as np
    from sklearn.model_selection import train_test_split
    X = df_compas_encoded.drop(['two_year_recid'], axis=1)
    y = df_compas_encoded['two_year_recid']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #print(X_test.columns)
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(np.vstack(X_train.values).astype(np.float64), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(np.vstack(X_test.values).astype(np.float64), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
    input_size = X_train_tensor.shape[1]
    output_size = 2
    fairness_to_select = 1
  else :

    from sklearn.datasets import fetch_openml
    from sklearn.preprocessing import LabelEncoder

    import numpy as np
    import pandas as pd

    # Fetch UCI Adult Income Dataset
    data = fetch_openml("adult", version=2, as_frame=True)
    df = data.frame
    #print((df.columns))

    # Encode target labels ('<=50K' or '>50K')
    label_encoder = LabelEncoder()
    df['income'] = label_encoder.fit_transform(df['class'])
    #print((df['income']))

    import torch
    import torch.nn as nn
    import numpy as np
    from sklearn.model_selection import train_test_split
    # Convert boolean columns to integers (0 for False, 1 for True)
    df = df.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

    # The rest remains unchanged

    df = pd.get_dummies(df, drop_first=True)
    df = df.fillna(0)

    X = df.drop(['income', 'class_>50K'], axis=1)
    y = df['income']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(np.vstack(X_train.values).astype(np.float32), dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(np.vstack(X_test.values).astype(np.float32), dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

    input_size = X_train_tensor.shape[1]
    output_size = 2
    fairness_to_select = 0
  return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, X_test, input_size, output_size, fairness_to_select

In [3]:
import torch
import torch.nn.functional as F


def fgsm_attack(model, data, target, epsilon):
    """Generates adversarial examples using FGSM."""
    data.requires_grad = True
    output = model(data)
    loss = F.binary_cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    perturbed_data = data + epsilon * data.grad.sign()
    return torch.clamp(perturbed_data, 0, 1)

def adversarial_train(model, X_train_tensor, y_train_tensor, optimizer, device, epsilon=0.1, attack_type='fgsm'):
    """Performs adversarial training using FGSM."""

    model.train()
    epochs = 10
    for epoch in range(epochs):
      epoch_loss = 0.0
      X_train_tensor, y_train_tensor = X_train_tensor.to(device), y_train_tensor.to(device)
      y_train_tensor = y_train_tensor.view(-1, 1)
      # Generate adversarial examples
      if attack_type == 'fgsm':
          adv_data = fgsm_attack(model, X_train_tensor, y_train_tensor, epsilon)
      else:
          adv_data = X_train_tensor  # Placeholder for PGD (can be implemented later)

      # Combine clean and adversarial examples
      mixed_data = torch.cat([X_train_tensor, adv_data])
      mixed_target = torch.cat([y_train_tensor, y_train_tensor])

      # Forward pass
      optimizer.zero_grad()
      output = model(mixed_data)
      loss = F.binary_cross_entropy(output, mixed_target)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()

      print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")


In [9]:
from sklearn.metrics import accuracy_score, confusion_matrix
def compute_class_features(model, X, y, num_classes):
    class_features = {i: [] for i in range(num_classes)}

    with torch.no_grad():
        for i in range(len(X)):
            features = model.extract_features(X[i].unsqueeze(0))
            class_features[y[i].item()].append(features)

    # Compute mean feature representation for each class
    for cls in class_features:
        if class_features[cls]:  # Avoid empty class
            class_features[cls] = torch.mean(torch.stack(class_features[cls]), dim=0)

    return class_features

def cafa_attack(model, X, y, target_class_features, epsilon=0.1):
    X_adv = X.clone().detach().requires_grad_(True)

    # Get original feature representation
    features = model.extract_features(X_adv)

    # Compute the feature shift direction towards target class
    target_features = target_class_features[y.item()]
    loss = torch.nn.functional.mse_loss(features, target_features)

    # Compute gradient and generate adversarial example
    loss.backward()
    X_adv = X_adv - epsilon * X_adv.grad.sign()
    return X_adv.detach()

def compute_accuracy(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        #_, predictions = torch.max(outputs, 1)

    #return accuracy_score(y_test.numpy(), predictions.numpy())
    return ((outputs > 0.5).float() == y_test).float().mean()

import torch
import torch.nn as nn

def pgd_adversarial_attack(model, X, y, epsilon=1, alpha=0.01, num_iter=40):
    """
    Projected Gradient Descent (PGD) attack.

    Args:
        model: The target model to attack.
        X: Input data (requires gradients).
        y: Correct labels for the input data.
        epsilon: Maximum allowed perturbation per input dimension.
        alpha: Step size for each iteration.
        num_iter: Number of iterations for the attack.

    Returns:
        Adversarial examples generated from X.
    """
    # Clone input and set it to require gradients
    X_adv = X.clone().detach().requires_grad_(True)
    loss_fn = nn.BCELoss()

    for _ in range(num_iter):
        # Zero the gradients
        model.zero_grad()

        # Forward pass and compute loss
        output = model(X_adv)
        loss = loss_fn(output, y)
        loss.backward()

        # Update adversarial examples with the gradient sign
        perturbation = alpha * X_adv.grad.sign()
        X_adv = X_adv + perturbation

        # Project back to the epsilon ball around the original inputs
        X_adv = torch.max(torch.min(X_adv, X + epsilon), X - epsilon)
        X_adv = torch.clamp(X_adv, 0, 1)  # Keep pixel values between 0 and 1

        # Reset gradient computation
        X_adv = X_adv.detach().requires_grad_(True)

    return X_adv

def fgsm_attack(model, X, y, epsilon):
    X.requires_grad_()  # Ensure gradients are computed
    model.zero_grad()  # Clear previous gradients
    output = model(X)
    loss = nn.BCELoss()(output, y)
    loss.backward()
    perturbation = epsilon * X.grad.sign()
    X_adv = torch.clamp(X + perturbation, 0, 1)
    return X_adv
def adversarial_attack_test_data(model, X, y, attack_name):
  if attack_name == 'PGD':
    X_adv = pgd_adversarial_attack(model, X, y)
  else:
    X_adv = fgsm_attack(model, X, y, 0.05)
  return X_adv
def adversarial_attack_accuracy(model, X, y, attack_name):
  if attack_name == 'PGD':
    X_adv = pgd_adversarial_attack(model, X, y)
    rslt = compute_accuracy(model, X_adv, y)
  else:
    X_adv = fgsm_attack(model, X, y, 0.05)
    rslt = compute_accuracy(model, X_adv, y)
  return rslt
from sklearn.metrics import confusion_matrix

def compute_tpr_fpr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    # Compute TPR and FPR
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

    return tpr, fpr
import torch
import torch.nn as nn
import numpy as np

def compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, sensitive_feature):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        predictions = (outputs > 0.5).float()  # Assuming binary classification with threshold 0.5

    # Convert sensitive_feature to torch tensor if it isn't already
    sensitive_feature_tensor = torch.tensor(sensitive_feature.values, dtype=torch.int32)

    # Calculate true positive rates (TPR) and false positive rates (FPR) for each group
    tpr_group_0 = ((predictions[sensitive_feature_tensor == 0] == 1) & (y_test_tensor[sensitive_feature_tensor == 0] == 1)).float().mean().item()
    tpr_group_1 = ((predictions[sensitive_feature_tensor == 1] == 1) & (y_test_tensor[sensitive_feature_tensor == 1] == 1)).float().mean().item()

    fpr_group_0 = ((predictions[sensitive_feature_tensor == 0] == 1) & (y_test_tensor[sensitive_feature_tensor == 0] == 0)).float().mean().item()
    fpr_group_1 = ((predictions[sensitive_feature_tensor == 1] == 1) & (y_test_tensor[sensitive_feature_tensor == 1] == 0)).float().mean().item()

    # Compute equalized odds difference
    tpr_difference = abs(tpr_group_0 - tpr_group_1)
    fpr_difference = abs(fpr_group_0 - fpr_group_1)

    equalized_odds_difference = max(tpr_difference, fpr_difference)

    return tpr_difference, fpr_difference

# Example usage
# Assuming you have a model, test data, and sensitive feature
# model = ...
# X_test_tensor = ...
# y_test_tensor = ...
# sensitive_feature = ...

# equalized_odds_diff = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, sensitive_feature)
# print("Equalized Odds Difference:", equalized_odds_diff)



In [23]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
def calculate_metrics_sk(model, X_test_tensor, y_test_tensor):
    # Convert predictions to binary (0 or 1) using a threshold of 0.5
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
    y_pred = (outputs > 0.5).float()

    y_true_np = y_test_tensor.numpy()
    y_pred_np = y_pred.numpy()
    precision = precision_score(y_true_np, y_pred_np)
    recall = recall_score(y_true_np, y_pred_np)
    f1 = f1_score(y_true_np, y_pred_np)
    return precision, recall, f1
def calculate_metrics(model, X_test_tensor, y_test_tensor):
    # Convert predictions to binary (0 or 1) using a threshold of 0.5
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
    y_pred = (outputs > 0.5).float()
    y_true = y_test_tensor
    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
    TP = ((y_pred == 1) & (y_true == 1)).sum().item()
    FP = ((y_pred == 1) & (y_true == 0)).sum().item()
    FN = ((y_pred == 0) & (y_true == 1)).sum().item()

    # Calculate Precision, Recall, and F1-Score
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1_score


In [5]:
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,X_test, input_size, output_size, fairness_to_select = select_dataset('compas')
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)
sensetive_features = [['sex_Male', 'race_Black', 'race_White'], ['sex',  'race_African-American', 'race_Asian']]

In [48]:
model = FFN(input_size, [10, 8], 2)

In [49]:
train_model(model, X_train_tensor, y_train_tensor, 20, 0.014043171929277431)

Epoch 1/20, Loss: 190.5865
Epoch 2/20, Loss: 29.3727
Epoch 3/20, Loss: 24.3434
Epoch 4/20, Loss: 24.4862
Epoch 5/20, Loss: 21.9655
Epoch 6/20, Loss: 22.5664
Epoch 7/20, Loss: 21.2915
Epoch 8/20, Loss: 21.0440
Epoch 9/20, Loss: 21.2390
Epoch 10/20, Loss: 20.9917
Epoch 11/20, Loss: 21.5201
Epoch 12/20, Loss: 20.9377
Epoch 13/20, Loss: 21.1960
Epoch 14/20, Loss: 18.9735
Epoch 15/20, Loss: 19.8763
Epoch 16/20, Loss: 18.1009
Epoch 17/20, Loss: 21.4593
Epoch 18/20, Loss: 19.3664
Epoch 19/20, Loss: 23.9865
Epoch 20/20, Loss: 19.3329


In [50]:
print(compute_accuracy(model, X_test_tensor, y_test_tensor))
print(calculate_metrics_sk(model, X_test_tensor, y_test_tensor))

tensor(0.9764)
(0.9621451104100947, 0.9838709677419355, 0.9728867623604466)


In [51]:
X_adv = adversarial_attack_test_data(model, X_test_tensor, y_test_tensor, 'FGSM')
print(compute_accuracy(model, X_adv, y_test_tensor))
print(calculate_metrics_sk(model, X_adv, y_test_tensor))

tensor(0.4297)
(0.42966042966042967, 1.0, 0.6010664081434803)


In [52]:
#model = FFN(input_size, [9, 10, 5], 2)
adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.1, attack_type='pgd')
adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')



Epoch 1/10, Loss: 0.0961
Epoch 2/10, Loss: 0.0924
Epoch 3/10, Loss: 0.0902
Epoch 4/10, Loss: 0.0892
Epoch 5/10, Loss: 0.0891
Epoch 6/10, Loss: 0.0894
Epoch 7/10, Loss: 0.0901
Epoch 8/10, Loss: 0.0904
Epoch 9/10, Loss: 0.0901
Epoch 10/10, Loss: 0.0895
Epoch 1/10, Loss: 1.7542
Epoch 2/10, Loss: 1.7443
Epoch 3/10, Loss: 1.7349
Epoch 4/10, Loss: 1.7254
Epoch 5/10, Loss: 1.7157
Epoch 6/10, Loss: 1.7061
Epoch 7/10, Loss: 1.6966
Epoch 8/10, Loss: 1.6872
Epoch 9/10, Loss: 1.6777
Epoch 10/10, Loss: 1.6684


In [54]:
print(adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM'))
adv_test = adversarial_attack_test_data(model, X_test_tensor, y_test_tensor, 'FGSM')
print(calculate_metrics_sk(model, adv_test, y_test_tensor))

tensor(0.4297)
(0.42966042966042967, 1.0, 0.6010664081434803)


tensor(0.5703)

In [81]:
adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')

tensor(0.2571)

In [55]:
!pip install pymoo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.5 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=39d634814fb03a1765c1d6524ffbd2f45079fd3162fa5038e10a7ee58713fc2e
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


In [85]:

import numpy as np
import torch
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.core.variable import Real
from pymoo.termination import get_termination
import random
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.core.callback import Callback
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

class FFNProblem(Problem):
    def __init__(self, max_layers, max_nodes, X_train, X_test, y_train, y_test):
        super().__init__(n_var=max_layers+2,
                         n_obj=4,
                         n_constr=1,
                         xl=5,
                         xu=max_nodes,
                         type_var=int)
        self.max_layers = max_layers
        self.max_nodes = max_nodes
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.count = 0
    def _evaluate(self, x, out, *args, **kwargs):
        adversarial_accuracy_PGD = np.zeros(x.shape[0])
        adversarial_accuracy_FGSM = np.zeros(x.shape[0])
        equalized_odds_gender = np.zeros(x.shape[0])
        equalized_odds_race_Black = np.zeros(x.shape[0])
        equalized_odds_race_White = np.zeros(x.shape[0])
        self.count += 1
        print(self.count)
        print(x)
        for i, architecture in enumerate(x):
            l, learning_rate_and_ind_opt = architecture[:-2], architecture[-2:]


            learning_rate = learning_rate_and_ind_opt[0]
            ind_opt = int(learning_rate_and_ind_opt[1])
            lr = 10 ** (-1*float(learning_rate))
            print(lr, ind_opt)
            l = np.round(l).astype(int)
            print(l)
            temp = l.tolist()
            print(input_size)
            model = FFN(input_size, temp, 2)

            #train_model(model, X_train_tensor, y_train_tensor, 5, lr, ind_opt)
            adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='pgd')
            #adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')

            #adv_acc, eq_odds = evaluate_model(architecture, self.X_train, self.X_test, self.y_train, self.y_test)
            accuracy_pgd = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')
            accuracy_fgsm = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
            tpr_diff_gender, fpr_diff_gender = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][0]])
            tpr_diff_race_Black, fpr_diff_race_Black = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][1]])
            tpr_diff_race_White, fpr_diff_race_White = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][2]])

            equalized_odd_gender = (tpr_diff_gender + fpr_diff_gender) / 2
            equalized_odd_race_Black = (tpr_diff_race_Black + fpr_diff_race_Black) / 2
            equalized_odd_race_White = (tpr_diff_race_White + fpr_diff_race_White) / 2

            adversarial_accuracy_PGD[i] = accuracy_pgd
            adversarial_accuracy_FGSM[i] = accuracy_fgsm

            equalized_odds_gender[i] = equalized_odd_gender
            equalized_odds_race_Black[i] = equalized_odd_race_Black
            equalized_odds_race_White[i] = equalized_odd_race_White
        out["G"] = np.column_stack([ 0.4- adversarial_accuracy_PGD])
        out["F"] = np.column_stack([adversarial_accuracy_PGD, equalized_odds_gender, equalized_odds_race_Black, equalized_odds_race_White])


# Initialize the problem
max_layers = 2  # Example: maximum number of layers
max_nodes = 50  # Example: maximum number of nodes per layer
problem = FFNProblem(max_layers, max_nodes, X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor)

# Define crossover and mutation operators
crossover = SBX(prob=0.9, eta=15)
mutation = PM(eta=20)

# Initialize the algorithm
#algorithm = NSGA2(pop_size=10, sampling=np.array(population))

l = []

for _ in range(4):
  pp = []
  for i in range(4):
    if i > 1:
      pp.append(random.randint(1, 7))
    else :

      pp.append(random.randint(5, 50))

  l.append(pp)
initial_population = np.array(l)
# Initialize the algorithm with the custom initial population
algorithm = NSGA2(
    pop_size=4,
    sampling = initial_population,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True
)

# Set the initial population



# Define the termination criterion
termination = get_termination("n_gen", 10)

# Run the optimization
res = minimize(problem,
               algorithm,
               termination,
               verbose=True)

# Print the results
print()
print("Best G for PGD found: %s" % res.G)
print()
print("Best solution found: %s" % res.F)
print()
print("Best architecture found: %s" % res.X)

1
[[11 27  7  3]
 [43 21  7  1]
 [34 39  1  4]
 [10 40  3  7]]
1e-07 3
[11 27]
51
Epoch 1/10, Loss: 54.4100
Epoch 2/10, Loss: 54.4100
Epoch 3/10, Loss: 54.4100
Epoch 4/10, Loss: 54.4100
Epoch 5/10, Loss: 54.4100
Epoch 6/10, Loss: 54.4100
Epoch 7/10, Loss: 54.4100
Epoch 8/10, Loss: 54.4100
Epoch 9/10, Loss: 54.4100
Epoch 10/10, Loss: 54.4100
1e-07 1
[43 21]
51
Epoch 1/10, Loss: 13.7758
Epoch 2/10, Loss: 10.9944
Epoch 3/10, Loss: 9.2522
Epoch 4/10, Loss: 7.4100
Epoch 5/10, Loss: 5.8142
Epoch 6/10, Loss: 4.6107
Epoch 7/10, Loss: 3.4582
Epoch 8/10, Loss: 2.4733
Epoch 9/10, Loss: 1.8914
Epoch 10/10, Loss: 1.4796
0.1 4
[34 39]
51
Epoch 1/10, Loss: 54.4515
Epoch 2/10, Loss: 54.4170
Epoch 3/10, Loss: 54.4104
Epoch 4/10, Loss: 54.4100
Epoch 5/10, Loss: 54.4100
Epoch 6/10, Loss: 54.4100
Epoch 7/10, Loss: 54.4100
Epoch 8/10, Loss: 54.4100
Epoch 9/10, Loss: 54.4100
Epoch 10/10, Loss: 54.4100
0.001 7
[10 40]
51
Epoch 1/10, Loss: 44.9670
Epoch 2/10, Loss: 38.8739
Epoch 3/10, Loss: 30.5766
Epoch 4/10

/usr/local/lib/python3.11/dist-packages/pymoo/operators/crossover/sbx.py:47: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (eta + 1.0)))[mask]


Epoch 4/10, Loss: 6.2131
Epoch 5/10, Loss: 4.0676
Epoch 6/10, Loss: 2.2480
Epoch 7/10, Loss: 1.3069
Epoch 8/10, Loss: 2.1088
Epoch 9/10, Loss: 3.0633
Epoch 10/10, Loss: 3.0561
1e-07 5
[11 27]
51
Epoch 1/10, Loss: 22.3425
Epoch 2/10, Loss: 21.6446
Epoch 3/10, Loss: 21.4742
Epoch 4/10, Loss: 21.7775
Epoch 5/10, Loss: 20.7845
Epoch 6/10, Loss: 18.6372
Epoch 7/10, Loss: 14.6554
Epoch 8/10, Loss: 9.5269
Epoch 9/10, Loss: 5.9726
Epoch 10/10, Loss: 3.7047
1e-07 5
[43 20]
51
Epoch 1/10, Loss: 4.2713
Epoch 2/10, Loss: 4.0009
Epoch 3/10, Loss: 1.9564
Epoch 4/10, Loss: 2.7380
Epoch 5/10, Loss: 2.0853
Epoch 6/10, Loss: 1.1540
Epoch 7/10, Loss: 1.1896
Epoch 8/10, Loss: 1.4601
Epoch 9/10, Loss: 1.4847
Epoch 10/10, Loss: 1.3226
2.3356885102365108e-08 5
[34 39]
51
Epoch 1/10, Loss: 54.4100
Epoch 2/10, Loss: 54.4100
Epoch 3/10, Loss: 54.4100
Epoch 4/10, Loss: 54.4100
Epoch 5/10, Loss: 54.4100
Epoch 6/10, Loss: 54.4100
Epoch 7/10, Loss: 54.4100
Epoch 8/10, Loss: 54.4100
Epoch 9/10, Loss: 54.4100
Epoch 1

In [86]:

import numpy as np
import torch
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.core.variable import Real
from pymoo.termination import get_termination
import random
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.core.callback import Callback
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

class FFNProblem(Problem):
    def __init__(self, max_layers, max_nodes, X_train, X_test, y_train, y_test):
        super().__init__(n_var=max_layers+2,
                         n_obj=4,
                         n_constr=1,
                         xl=5,
                         xu=max_nodes,
                         type_var=int)
        self.max_layers = max_layers
        self.max_nodes = max_nodes
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.count = 0
    def _evaluate(self, x, out, *args, **kwargs):
        adversarial_accuracy_PGD = np.zeros(x.shape[0])
        adversarial_accuracy_FGSM = np.zeros(x.shape[0])
        equalized_odds_gender = np.zeros(x.shape[0])
        equalized_odds_race_Black = np.zeros(x.shape[0])
        equalized_odds_race_White = np.zeros(x.shape[0])
        self.count += 1
        print(self.count)
        print(x)
        for i, architecture in enumerate(x):
            l, learning_rate_and_ind_opt = architecture[:-2], architecture[-2:]


            learning_rate = learning_rate_and_ind_opt[0]
            ind_opt = int(learning_rate_and_ind_opt[1])
            lr = 10 ** (-1*float(learning_rate))
            print(lr, ind_opt)
            l = np.round(l).astype(int)
            print(l)
            temp = l.tolist()
            print(input_size)
            model = FFN(input_size, temp, 2)

            #train_model(model, X_train_tensor, y_train_tensor, 5, lr, ind_opt)
            #adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='pgd')
            adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')

            #adv_acc, eq_odds = evaluate_model(architecture, self.X_train, self.X_test, self.y_train, self.y_test)
            accuracy_pgd = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')
            accuracy_fgsm = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
            tpr_diff_gender, fpr_diff_gender = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][0]])
            tpr_diff_race_Black, fpr_diff_race_Black = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][1]])
            tpr_diff_race_White, fpr_diff_race_White = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][2]])

            equalized_odd_gender = (tpr_diff_gender + fpr_diff_gender) / 2
            equalized_odd_race_Black = (tpr_diff_race_Black + fpr_diff_race_Black) / 2
            equalized_odd_race_White = (tpr_diff_race_White + fpr_diff_race_White) / 2

            #adversarial_accuracy_PGD[i] = accuracy_pgd
            adversarial_accuracy_FGSM[i] = accuracy_fgsm

            equalized_odds_gender[i] = equalized_odd_gender
            equalized_odds_race_Black[i] = equalized_odd_race_Black
            equalized_odds_race_White[i] = equalized_odd_race_White
        out["G"] = np.column_stack([ 0.4- adversarial_accuracy_FGSM])
        out["F"] = np.column_stack([ adversarial_accuracy_FGSM, equalized_odds_gender, equalized_odds_race_Black, equalized_odds_race_White])

random.seed(42)
# Initialize the problem
max_layers = 2  # Example: maximum number of layers
max_nodes = 50  # Example: maximum number of nodes per layer
problem = FFNProblem(max_layers, max_nodes, X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor)

# Define crossover and mutation operators
crossover = SBX(prob=0.9, eta=15)
mutation = PM(eta=20)

# Initialize the algorithm
#algorithm = NSGA2(pop_size=10, sampling=np.array(population))

l = []

for _ in range(4):
  pp = []
  for i in range(4):
    if i > 1:
      pp.append(random.randint(1, 7))
    else :

      pp.append(random.randint(5, 50))

  l.append(pp)
initial_population = np.array(l)
# Initialize the algorithm with the custom initial population
algorithm = NSGA2(
    pop_size=4,
    sampling = initial_population,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True
)

# Set the initial population



# Define the termination criterion
termination = get_termination("n_gen", 10)

# Run the optimization
res = minimize(problem,
               algorithm,
               termination,
               verbose=True)

# Print the results
print()
print("Best G for FGSM found: %s" % res.G)
print()
print("Best solution found: %s" % res.F)
print()
print("Best architecture found: %s" % res.X)

1
[[45 12  1  6]
 [22 20  2  2]
 [11 48  6  5]
 [10 42  4  1]]
0.1 6
[45 12]
51
Epoch 1/10, Loss: 17.0410
Epoch 2/10, Loss: 10.9655
Epoch 3/10, Loss: 6.8667
Epoch 4/10, Loss: 4.2791
Epoch 5/10, Loss: 2.9333
Epoch 6/10, Loss: 2.2921
Epoch 7/10, Loss: 1.9039
Epoch 8/10, Loss: 1.6350
Epoch 9/10, Loss: 1.3794
Epoch 10/10, Loss: 1.1816
0.01 2
[22 20]
51
Epoch 1/10, Loss: 27.5551
Epoch 2/10, Loss: 27.5538
Epoch 3/10, Loss: 27.5528
Epoch 4/10, Loss: 27.5520
Epoch 5/10, Loss: 27.5513
Epoch 6/10, Loss: 27.5507
Epoch 7/10, Loss: 27.5428
Epoch 8/10, Loss: 27.4974
Epoch 9/10, Loss: 26.8492
Epoch 10/10, Loss: 25.2673
1e-06 5
[11 48]
51
Epoch 1/10, Loss: 12.1170
Epoch 2/10, Loss: 9.1987
Epoch 3/10, Loss: 6.2171
Epoch 4/10, Loss: 3.9654
Epoch 5/10, Loss: 3.6517
Epoch 6/10, Loss: 7.6394
Epoch 7/10, Loss: 8.3638
Epoch 8/10, Loss: 5.4471
Epoch 9/10, Loss: 2.9540
Epoch 10/10, Loss: 2.0187
0.0001 1
[10 42]
51
Epoch 1/10, Loss: 27.5590
Epoch 2/10, Loss: 27.5576
Epoch 3/10, Loss: 27.5563
Epoch 4/10, Loss: 2

/usr/local/lib/python3.11/dist-packages/pymoo/operators/crossover/sbx.py:47: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (eta + 1.0)))[mask]


Epoch 4/10, Loss: 4.0281
Epoch 5/10, Loss: 1.4864
Epoch 6/10, Loss: 1.2816
Epoch 7/10, Loss: 1.4832
Epoch 8/10, Loss: 1.5573
Epoch 9/10, Loss: 1.5009
Epoch 10/10, Loss: 1.3830
5.880139907956525e-09 5
[10 48]
51
Epoch 1/10, Loss: 27.7689
Epoch 2/10, Loss: 27.6333
Epoch 3/10, Loss: 27.5760
Epoch 4/10, Loss: 27.5585
Epoch 5/10, Loss: 27.5530
Epoch 6/10, Loss: 27.5508
Epoch 7/10, Loss: 27.5494
Epoch 8/10, Loss: 27.5485
Epoch 9/10, Loss: 27.5478
Epoch 10/10, Loss: 27.5471
1e-06 5
[44 47]
51
Epoch 1/10, Loss: 14.6961
Epoch 2/10, Loss: 7.8404
Epoch 3/10, Loss: 3.4561
Epoch 4/10, Loss: 9.8610
Epoch 5/10, Loss: 14.0471
Epoch 6/10, Loss: 12.4407
Epoch 7/10, Loss: 7.9137
Epoch 8/10, Loss: 3.5361
Epoch 9/10, Loss: 2.2788
Epoch 10/10, Loss: 2.2096
1e-05 7
[ 7 38]
51
Epoch 1/10, Loss: 27.5540
Epoch 2/10, Loss: 27.5500
Epoch 3/10, Loss: 27.5491
Epoch 4/10, Loss: 27.5487
Epoch 5/10, Loss: 27.5484
Epoch 6/10, Loss: 27.5481
Epoch 7/10, Loss: 27.5478
Epoch 8/10, Loss: 27.5474
Epoch 9/10, Loss: 27.5471
Ep

**Below code Envolved with weight search space : **

In [65]:
def mutate(ffn, mutation_rate=0.1, mutation_strength=0.01):
    parameters = ffn.get_parameters()
    for i in range(len(parameters)):
        if np.random.rand() < mutation_rate:
            noise = torch.randn_like(parameters[i]) * mutation_strength
            parameters[i] += noise
    ffn.set_parameters(parameters)


In [100]:
population_size = res.X.shape[0]
population_structures = []
for i in range(population_size):
    population_structures.append(res.X[0].tolist())



In [101]:
from copy import deepcopy
# Train 5 initial models on the COMPAS dataset
trained_models = []
for _ in population_structures:

    l = []
    for i in range(len(_)-2):
      l.append(int(_[i]))


    model = FFN(input_size, l, 2)
    adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=0.001), 'cpu', epsilon=0.2, attack_type='fgsm')
    #train_model(model, X_train_tensor, y_train_tensor)  # Assume this function trains the model
    trained_models.append(model)

# Clone and mutate to complete the population

population = []
for _ in range(population_size):
    base_model = deepcopy(trained_models[np.random.randint(population_size)])
    mutate(base_model)  # Apply slight mutation for diversity
    population.append(base_model)


Epoch 1/10, Loss: 18.2089
Epoch 2/10, Loss: 14.6884
Epoch 3/10, Loss: 10.4617
Epoch 4/10, Loss: 8.5527
Epoch 5/10, Loss: 5.3944
Epoch 6/10, Loss: 3.8625
Epoch 7/10, Loss: 2.4024
Epoch 8/10, Loss: 2.8767
Epoch 9/10, Loss: 1.4216
Epoch 10/10, Loss: 1.1445
Epoch 1/10, Loss: 5.2481
Epoch 2/10, Loss: 15.8512
Epoch 3/10, Loss: 16.9626
Epoch 4/10, Loss: 16.4212
Epoch 5/10, Loss: 15.0354
Epoch 6/10, Loss: 12.5992
Epoch 7/10, Loss: 8.9510
Epoch 8/10, Loss: 5.1993
Epoch 9/10, Loss: 4.2400
Epoch 10/10, Loss: 4.4611
Epoch 1/10, Loss: 4.0013
Epoch 2/10, Loss: 4.2940
Epoch 3/10, Loss: 7.3669
Epoch 4/10, Loss: 3.1384
Epoch 5/10, Loss: 0.9733
Epoch 6/10, Loss: 1.1895
Epoch 7/10, Loss: 1.3131
Epoch 8/10, Loss: 1.1026
Epoch 9/10, Loss: 0.8833
Epoch 10/10, Loss: 0.9559
Epoch 1/10, Loss: 5.9092
Epoch 2/10, Loss: 3.1716
Epoch 3/10, Loss: 9.2428
Epoch 4/10, Loss: 6.6872
Epoch 5/10, Loss: 2.2299
Epoch 6/10, Loss: 1.2813
Epoch 7/10, Loss: 1.6863
Epoch 8/10, Loss: 1.7675
Epoch 9/10, Loss: 1.5249
Epoch 10/10, L

In [102]:
[sum(param.numel() for param in population[i].get_parameters()) for i in range(len(population))]

[4397, 4397, 4397, 4397]

In [53]:
population[0].parameters()

<generator object Module.parameters at 0x7e76bf3a4200>

In [54]:
for i, param in enumerate(population[0].get_parameters()):
  print(i)
  print(param[0].shape)

0
torch.Size([97])
1
torch.Size([])
2
torch.Size([6])
3
torch.Size([])
4
torch.Size([7])
5
torch.Size([])
6
torch.Size([6])
7
torch.Size([])
8
torch.Size([8])
9
torch.Size([])
10
torch.Size([6])
11
torch.Size([])


In [103]:
import numpy as np
import torch
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.callback import Callback


# Define the optimization problem
class FFNOptimizationProblem(Problem):
    def __init__(self, model):
        self.model = model


        # Flatten the parameters to create a single vector of variables
        self.original_params = model.get_parameters()
        self.param_shapes = [param.shape for param in self.original_params]
        self.num_vars = sum(param.numel() for param in self.original_params)

        super().__init__(n_var=self.num_vars, n_obj=5, n_constr=0, xl=-1.0, xu=1.0)

    def _evaluate(self, x, out, *args, **kwargs):
        adversarial_accuracy_PGD = np.zeros(x.shape[0])
        adversarial_accuracy_FGSM = np.zeros(x.shape[0])
        equalized_odds_gender = np.zeros(x.shape[0])
        equalized_odds_race_Black = np.zeros(x.shape[0])
        equalized_odds_race_White = np.zeros(x.shape[0])
        # Reshape the flat vector back into the model parameters
        for i in range(x.shape[0]):  # Iterate over each individual in the population
            params = self._reshape_params(x[i])
            self.model.set_parameters(params)
            model = self.model
            accuracy_pgd = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'PGD')
            accuracy_fgsm = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
            tpr_diff_gender, fpr_diff_gender = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][0]])
            tpr_diff_race_Black, fpr_diff_race_Black = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][1]])
            tpr_diff_race_White, fpr_diff_race_White = compute_equalized_odds_difference(model, X_test_tensor, y_test_tensor, X_test[sensetive_features[fairness_to_select][2]])

            equalized_odd_gender = (tpr_diff_gender + fpr_diff_gender) / 2
            equalized_odd_race_Black = (tpr_diff_race_Black + fpr_diff_race_Black) / 2
            equalized_odd_race_White = (tpr_diff_race_White + fpr_diff_race_White) / 2

            adversarial_accuracy_PGD[i] = accuracy_pgd
            adversarial_accuracy_FGSM[i] = accuracy_fgsm

            equalized_odds_gender[i] = equalized_odd_gender
            equalized_odds_race_Black[i] = equalized_odd_race_Black
            equalized_odds_race_White[i] = equalized_odd_race_White

        out["F"] = np.column_stack([adversarial_accuracy_PGD, adversarial_accuracy_FGSM, equalized_odds_gender, equalized_odds_race_Black, equalized_odds_race_White])





    def _reshape_params(self, x):
        params = []
        index = 0
        for shape in self.param_shapes:
            size = np.prod(shape)
            params.append(torch.tensor(x[index:index + size].reshape(shape), dtype=torch.float32))
            index += size
        return params


model = population[0]


# Load pretrained models (example)
pretrained_models = population
# Assume pretrained_models are already trained and parameters are set

# Flatten parameters of pretrained models to use as initial population
initial_population = []
expected_size = sum(p.numel() for p in pretrained_models[0].get_parameters())
for pretrained_model in pretrained_models:

    params = pretrained_model.get_parameters()
    flat_params = np.concatenate([p.detach().cpu().numpy().flatten() for p in params])
    print(f"Model {i} flattened parameter shape: {flat_params.shape}")
    if flat_params.shape[0] != expected_size:
        print(f"Model {i} has an inconsistent shape: {flat_params.shape}")
    initial_population.append(flat_params)
# Convert initial population to a numpy array
initial_population = np.stack(initial_population)

# Define the problem
problem = FFNOptimizationProblem(model)

# Initialize the algorithm with the initial population
algorithm = NSGA2(pop_size=4, sampling=initial_population)

# Run the optimization
res = minimize(problem, algorithm, ('n_gen', 5), verbose=True)

print("Best solution found: %s" % res.F[0])
#print("Best variables found: %s" % res.X[0])

Model 1 flattened parameter shape: (4397,)
Model 1 flattened parameter shape: (4397,)
Model 1 flattened parameter shape: (4397,)
Model 1 flattened parameter shape: (4397,)
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        3 |      3 |             - |             -
     2 |        7 |      4 |  0.8351783901 |         ideal
     3 |       11 |      4 |  0.3564867807 |         ideal
     4 |       15 |      4 |  0.7975205099 |         ideal
     5 |       19 |      4 |  0.2054541450 |             f
Best solution found: [5.70339561e-01 5.70339561e-01 4.19111486e-04 6.83994556e-04
 3.47705150e-04]


In [45]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00


In [104]:
import optuna

def objective(trial):
    # Suggest hyperparameters
    hidden_sizes = [trial.suggest_int('n_units_l1', 5, 50),
                    trial.suggest_int('n_units_l2', 5, 50)]

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Define the model
    model = FFN(input_size=input_size, hidden_sizes=hidden_sizes, output_size=2)


    train_model(model, X_train_tensor, y_train_tensor)
    adversarial_train(model, X_train_tensor, y_train_tensor, optim.Adam(model.parameters(), lr=learning_rate), 'cpu', epsilon=0.2, attack_type='fgsm')
    # Evaluation
    accuracy = adversarial_attack_accuracy(model, X_test_tensor, y_test_tensor, 'FGSM')
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
tt = []
for key, value in trial.params.items():
    tt.append(value)
    print("    {}: {}".format(key, value))
lerr = tt[-1]
tt = tt[: -1]

[I 2025-02-15 02:41:28,307] A new study created in memory with name: no-name-c16ae3c3-bf26-4290-979f-09f67dcdd1d8
<ipython-input-104-22c42653ff17>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


Epoch 1/10, Loss: 9848.2955
Epoch 2/10, Loss: 9854.2614
Epoch 3/10, Loss: 9854.2614
Epoch 4/10, Loss: 9848.2955
Epoch 5/10, Loss: 9836.3636
Epoch 6/10, Loss: 9854.2614
Epoch 7/10, Loss: 9860.2273
Epoch 8/10, Loss: 9848.2955
Epoch 9/10, Loss: 9830.3977


[I 2025-02-15 02:41:43,373] Trial 0 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 19, 'n_units_l2': 24, 'learning_rate': 0.002264229580328559}. Best is trial 0 with value: 0.429660439491272.


Epoch 10/10, Loss: 9836.3636
Epoch 1/10, Loss: 27.5648
Epoch 2/10, Loss: 27.5623
Epoch 3/10, Loss: 27.5602
Epoch 4/10, Loss: 27.5585
Epoch 5/10, Loss: 27.5570
Epoch 6/10, Loss: 27.5484
Epoch 7/10, Loss: 27.3186
Epoch 8/10, Loss: 21.9705
Epoch 9/10, Loss: 12.4285
Epoch 10/10, Loss: 7.9320
Epoch 1/10, Loss: 9836.3859
Epoch 2/10, Loss: 9842.3295
Epoch 3/10, Loss: 9848.2955
Epoch 4/10, Loss: 9842.3295
Epoch 5/10, Loss: 9848.2955
Epoch 6/10, Loss: 9836.3636
Epoch 7/10, Loss: 9842.3295
Epoch 8/10, Loss: 9854.2614
Epoch 9/10, Loss: 9818.4659


[I 2025-02-15 02:41:58,351] Trial 1 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 11, 'n_units_l2': 5, 'learning_rate': 3.782291530871839e-05}. Best is trial 0 with value: 0.429660439491272.


...

Epoch 9/10, Loss: 0.4152
Epoch 10/10, Loss: 0.4148
Epoch 1/10, Loss: 1745.3432
Epoch 2/10, Loss: 55.4775
Epoch 3/10, Loss: 50.5195
Epoch 4/10, Loss: 45.9385
Epoch 5/10, Loss: 47.2589
Epoch 6/10, Loss: 35.8309
Epoch 7/10, Loss: 36.1129
Epoch 8/10, Loss: 33.2118
Epoch 9/10, Loss: 30.0671
Epoch 10/10, Loss: 23.7860
Epoch 1/10, Loss: 0.4059
Epoch 2/10, Loss: 1.0123
Epoch 3/10, Loss: 0.4175
Epoch 4/10, Loss: 0.7028
Epoch 5/10, Loss: 0.5465
Epoch 6/10, Loss: 0.4073
Epoch 7/10, Loss: 0.4331
Epoch 8/10, Loss: 0.4987


[I 2025-02-15 02:43:46,541] Trial 8 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 42, 'n_units_l2': 39, 'learning_rate': 0.0017280217925293908}. Best is trial 5 with value: 0.4442134499549866.


Epoch 9/10, Loss: 0.5264
Epoch 10/10, Loss: 0.4988
Epoch 1/10, Loss: 256.4975
Epoch 2/10, Loss: 39.0347
Epoch 3/10, Loss: 30.0177
Epoch 4/10, Loss: 30.3530
Epoch 5/10, Loss: 24.8702
Epoch 6/10, Loss: 24.2013
Epoch 7/10, Loss: 22.6400
Epoch 8/10, Loss: 24.7529
Epoch 9/10, Loss: 21.7610


[I 2025-02-15 02:44:01,794] Trial 9 finished with value: 0.429660439491272 and parameters: {'n_units_l1': 11, 'n_units_l2': 12, 'learning_rate': 0.00020228582828626333}. Best is trial 5 with value: 0.4442134499549866.


Epoch 10/10, Loss: 24.7731
Epoch 1/10, Loss: 0.4011
Epoch 2/10, Loss: 0.3957
Epoch 3/10, Loss: 0.3974
Epoch 4/10, Loss: 0.3982
Epoch 5/10, Loss: 0.3965
Epoch 6/10, Loss: 0.3952
Epoch 7/10, Loss: 0.3953
Epoch 8/10, Loss: 0.3959
Epoch 9/10, Loss: 0.3962
Epoch 10/10, Loss: 0.3957
Best trial:
  Value: 0.4442134499549866
  Params: 
    n_units_l1: 12
    n_units_l2: 18
    learning_rate: 0.004689196958165609
